# Add Forecasts

**Originally Contributed by**: Clayton Barrows

## Introduction

An example of how to parse add time series data to a `System` using [PowerSystems.jl](github.com/NREL/PowerSystems.jl)

For example, a `System` created by [parsing a MATPOWER file](../../notebook/PowerSystems.jl Examples/parse_matpower.ipynb)
doesn't contain any time series data. So a user may want to add forecasts to the `System`
### Dependencies
Let's use the 5-bus dataset we parsed in the MATPOWER example

In [1]:
using SIIPExamples
pkgpath = dirname(dirname(pathof(SIIPExamples)))
include(joinpath(pkgpath,"script/PowerSystems.jl Examples/parse_matpower.jl"))

Project SIIPExamples v0.0.1
    Status `~/Documents/repos/Examples/Project.toml`
  [9961bab8] Cbc v0.6.5
  [5ae59095] Colors v0.9.6
  [41994980] D3TypeTrees v0.1.1
  [a93c6f00] DataFrames v0.19.4
  [e2685f51] ECOS v0.10.0
  [60bf3e95] GLPK v0.12.0
  [b6b21f68] Ipopt v0.6.0
  [4076af6c] JuMP v0.20.0
  [51fcb6bd] NamedColors v0.2.0
  [f0f68f2c] PlotlyJS v0.13.0
  [91a5bcdd] Plots v0.27.0
  [e690365d] PowerSimulations v0.2.0 #jd/updates_to_canonical (https://github.com/NREL/PowerSimulations.jl.git)
  [bcd98974] PowerSystems v0.5.1 #master (https://github.com/NREL/PowerSystems.jl.git)
  [9e3dc215] TimeSeries v0.16.0
  [44d3d7a6] Weave v0.9.1
  [0f1e0344] WebIO v0.8.11


┌ Info: extending matpower format with data: areas 1x3
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/LKqny/src/parsers/pm_io/matpower.jl:290
┌ Info: extending matpower format with data: gen_name 7x4
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/LKqny/src/parsers/pm_io/matpower.jl:290
┌ Info: extending matpower format by appending matrix "gen_name" in to "gen"
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/LKqny/src/parsers/pm_io/matpower.jl:625
┌ Info: reversing the orientation of branch 6 (4, 3) to be consistent with other parallel branches
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/LKqny/src/parsers/pm_io/data.jl:1190
┌ Info: the voltage setpoint on generator 4 does not match the value at bus 4
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/LKqny/src/parsers/pm_io/data.jl:1656
┌ Info: the voltage setpoint on generator 1 does not match the value at bus 1
└ @ PowerSystems /Users/cbarrows/.julia/packages/Po

,ConcreteType,SuperTypes,Count
,String,String,Int64
1,Bus,Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,5
2,Line,ACBranch <: Branch <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,5
3,LoadZones,Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,1
4,PhaseShiftingTransformer,ACBranch <: Branch <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,2
5,PowerLoad,StaticLoad <: ElectricLoad <: Injection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,3
6,RenewableDispatch,RenewableGen <: Generator <: Injection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,2
7,ThermalStandard,ThermalGen <: Generator <: Injection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,5


### Define pointers to time series files
For example, if we want to add a bunch of time series files, say one for each load and 
one for each renewable generator, we need to define pointers to each .csv file cotaining 
the time series in the following format (PowerSystems.jl also supports a CSV format for this file)

In [2]:
FORECASTS_DIR = joinpath(base_dir,"data/forecasts/5bus_ts")
fname = joinpath(FORECASTS_DIR,"timeseries_pointers_da.json")
open(fname,"r") do f
    for line in eachline(f)
        println(line)
    end
end

[
    {
        "simulation": "DAY_AHEAD",
        "category": "Generator",
        "component_name": "SolarBusC",
        "label": "PMax MW",
        "scaling_factor": 1.0,
        "data_file": "./gen/Renewable/PV/da_solar5.csv"
    },
    {
        "simulation": "REAL_TIME",
        "category": "Generator",
        "component_name": "WindBusA",
        "label": "PMax MW",
        "scaling_factor": 1.0,
        "data_file": "./gen/Renewable/WIND/da_wind5.csv"
    },
    {
        "simulation": "REAL_TIME",
        "category": "ElectricLoad",
        "component_name": "bus2",
        "label": "MW Load",
        "scaling_factor": 1.0,
        "data_file": "./load/da_load5.csv"
    },
    {
        "simulation": "REAL_TIME",
        "category": "ElectricLoad",
        "component_name": "bus3",
        "label": "MW Load",
        "scaling_factor": 1.0,
        "data_file": "./load/da_load5.csv"
    },
    {
        "simulation": "REAL_TIME",
        "category": "ElectricLoad",
        "co

### Map labels to fields in `PowerSystemTypes`
next we need to define the fields that each time series file references. The above file
has two unique combinations of `category` and `label`, so we need to make a label mapping
to match each of those to the `component` fields.

In [ ]:
label_mapping = Dict(("electricload","MW Load") => "maxactivepower",
    ("generator","PMax MW") => ["get_tech","get_rating"])

### Read the pointers and label mappings

In [ ]:
ts_pointers = IS.read_time_series_metadata(fname, label_mapping)

### Read and assign forecasts to `System`

In [ ]:
add_forecasts!(sys, ts_pointers)
sys